# Tools

- Tools are utilities designed to be called by a model: their inputs are designed to be generated by models, and their outputs are designed to be passed back to models.

In [ ]:
!pip install langchain langchain-core langchain-community pydantic duckduckgo-search langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


## Built-in Tool - DuckDuckGo Search

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke('top news of IPL match')

print(results)

IPL 2025, MI vs GT Live Cricket Score Streaming Online, Pitch-Weather Forecast Report: Mumbai Indians take on the Gujarat Titans in a battle that will bolster the playoff credentials of one team and will weaken the claim of another team. Both teams come into this clash with 14 points, thanks to seven wins. Cricket News: With 15 IPL 2025 league stage games remaining, RCB, GT, and PBKS are almost certain for the playoffs, while MI is a strong contender. DC has a fair ch Cricket News: Heavy rain led to the washout of the IPL match between Sunrisers Hyderabad and Delhi Capitals, resulting in both teams receiving one point. This outco IPL 2025 SRH vs DC Highlights: The match has been called off due to heavy rain in Hyderabad. This after Delhi Capitals posted 133/7 in their 20 overs. DC suffered a top-order collapse, losing five wickets inside the first seven overs. However, late fightbacks from Tristan Stubbs (41) and Ashutosh Sharma (41 off 26) gave them something to bowl at. However, rain

In [ ]:
print(search_tool.name,end='\n\n')
print(search_tool.description,end='\n\n')
print(search_tool.args)

duckduckgo_search

A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.

{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


## Built-in Tool - Shell Tool

In [ ]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke('ls')

print(results)

Executing command:
 ls
sample_data



/usr/local/lib/python3.11/dist-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


## Custom Tools

In [ ]:
from langchain_core.tools import tool


# Step 1 - create a function

def multiply(a, b):
    """Multiply two numbers"""
    return a*b


# Step 2 - add type hints

def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b


# Step 3 - add tool decorator

@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b


result = multiply.invoke({"a":3, "b":5})
print(result)

15


In [ ]:
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.args_schema.model_json_schema())

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


## Method 2 - Using StructuredTool

In [ ]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [ ]:
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [ ]:
def multiply_func(a: int, b: int) -> int:
    return a * b

In [ ]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

In [ ]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


## Method 3 - Using BaseTool Class

In [ ]:
from langchain.tools import BaseTool
from typing import Type

In [ ]:
# arg schema using pydantic

class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [ ]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

In [ ]:
multiply_tool = MultiplyTool()

In [ ]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


## Toolkit

In [ ]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


In [ ]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]


In [ ]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)


add => Add two numbers
multiply => Multiply two numbers
